### 叔叔我啊，要开始训练了捏

首先，获取模型和数据


In [1]:
import os

import numpy as np
import torch
from tqdm import tqdm

from dataset import MyDataset
from model import LSTM, MyLSTM, MyTransformerEncoder

device = "cuda" if torch.cuda.is_available() else "cpu"

model = MyTransformerEncoder(d_model=100, output_dimension=1)
# model = MyLSTM(d_model=100, output_dimension=1)
# model = LSTM(256)
train_data = np.load("./npys/train/sentence_code.npy")
train_labels = np.load("./npys/train/labels.npy")
vocabulary_vectors = np.load("./npys/vocabulary_vectors.npy")
# dataset = MyDataset(train_data[12000:13000], train_labels[12000:13000], vocabulary_vectors)
dataset = MyDataset(train_data, train_labels, vocabulary_vectors)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=True) # Windows上，num_workers太高会慢，很sb


optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.995)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [2]:
def train(max_epoch):
    loss_fn = torch.nn.BCELoss().to(device)
    # loss_fn = torch.nn.CrossEntropyLoss().to(device)
    for epoch in range(max_epoch):
        avg_loss = []
        for X, y in tqdm(dataloader):
            optimizer.zero_grad()
            output = model(X)
            # print(X.shape)
            # print(output)
            # print(y)
            loss = loss_fn(output, y)
            avg_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            scheduler.step()
        if epoch%10==0:
            print("Epoch: %d | loss: %f"%(epoch, np.average(avg_loss)))

训练，保存：

In [3]:
dataset[0]

/Users/yyt/GitHub/mlpda-project/dataset.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)
  sentence = torch.as_tensor(sentence, dtype=torch.float32, device=self.device)


(tensor([[-0.1440,  0.3255,  0.1426,  ...,  0.2540,  1.1078, -0.0731],
         [-0.2709,  0.0440, -0.0203,  ..., -0.4923,  0.6369,  0.2364],
         [ 0.3825,  0.1482,  0.6060,  ...,  0.0589,  0.0911,  0.4728],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor(1.))

In [4]:
# train(500)
os.makedirs("./trained_models/", exist_ok=True)
torch.save(model, "./trained_models/transformer_full_%d"%500)